In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import confusion_matrix, mean_squared_error

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.losses import MeanSquaredError

In [ ]:
rel_data = pd.read_csv('/kaggle/input/reliance-stock-prediction/daily_adjusted_RELIANCE.BSE.csv')
rel_data.head()

In [ ]:
rel_data['timestamp'] = pd.to_datetime(rel_data['timestamp'])

In [ ]:
rel_data = rel_data[::-1]
rel_data

In [ ]:
rel_data = rel_data.reset_index().drop(['index'], axis=1)
rel_data

In [ ]:
rel_data.isnull().sum()

In [ ]:
plt.figure(figsize=(14,10))
plt.plot(rel_data['timestamp'], rel_data['close'])
plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Stock Value')
plt.show()

In [ ]:
features = [x for x in rel_data.columns if x not in ['split_coefficient', 'adjusted_close']]
rel_data = rel_data[features]

In [ ]:
train = rel_data[rel_data['timestamp'] < dt.datetime(2020, 7, 1)]
test = rel_data[rel_data['timestamp'] >= dt.datetime(2020, 7, 1)]

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))

In [ ]:
# Selecting only the 'open' value in the form of an array for the input to the model
train = train.iloc[:,1:2].values

In [ ]:
train_scaled = scaler.fit_transform(train)
# train_scaled.head()

In [ ]:
X_train = []
y_train = []
# Group size of 60
# Arrays stored in X_train as groups of 60.
for i in range(60,3812):
    X_train.append(train_scaled[i-60:i,0])
    y_train.append(train_scaled[i,0])


In [ ]:
X_train, y_train = np.array(X_train), np.array(y_train)
# Reshape the array for the model
# Adding one extra dimension for each value inside of the group.
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(X_train, y_train, epochs=60, batch_size=32)

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
inputs = rel_data[len(rel_data)-len(test)-60:]
inputs = inputs.iloc[:,1:2].values
inputs = inputs.reshape(-1,1)
inputs = scaler.transform(inputs)

In [ ]:
assert inputs.shape == (343,1)
# print(inputs.shape)
# print(test.shape)
# print(X_test.shape)

In [ ]:
X_test = []
y_test = []
for i in range(60, 343):
    X_test.append(inputs[i-60:i,0])
    y_test.append(inputs[i,0])
X_test, y_test = np.array(X_test), np.array(y_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


In [ ]:
preds = model.predict(X_test)
preds = scaler.inverse_transform(preds)

In [ ]:
mse = MeanSquaredError()
mse(preds, y_test).numpy()

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(test.iloc[:,1:2].values, color = 'green', label = 'Reliance Stock Price')
plt.plot(preds, color = 'red', label = 'Predicted Reliance Stock Price')
plt.title('Stock Price Prediction')
plt.xlabel('Trading Day')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(rel_data[rel_data['timestamp'] < dt.datetime(2020, 7, 1)]['timestamp'] ,train)
plt.plot(rel_data[rel_data['timestamp'] >= dt.datetime(2020, 7, 1)]['timestamp'], test.iloc[:,1:2].values, color = 'green', label = 'Reliance Stock Price')
plt.plot(rel_data[rel_data['timestamp'] >= dt.datetime(2020, 7, 1)]['timestamp'], preds, color = 'red', label = 'Predicted Reliance Stock Price')
plt.title('Stock Price Prediction')
plt.xlabel('Trading Day')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

In [ ]:
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)

In [ ]:
import h5py

In [ ]:
model.save_weights('model.h5')